In [28]:
import pathlib
from Bio import SeqIO
import pandas as pd
import pickle
import numpy as np
import multiprocessing as mp

# Try to create binning code interactively - 
## i1 is the intermediate results right after classify()   i2 is the results at the end.

### Notes for this part: 
1. If I'm gonna use a dataframe and all that might as well make the columns and format it prior to binning.
2. If not make sure to think of alternative.

In [3]:
# Define output folder from a Strainr run

output_folder = pathlib.Path('howdy4/')

# Extract intermediate from pkl
i1 = output_folder / 'i1.pkl'
with i1.open('rb') as ph:
    dfp1 = pickle.load(ph)

# Extract final from pickle
i2 = output_folder / 'i2.pkl'
with i2.open('rb') as ph:
    dfp2 = pickle.load(ph)

# Get the strain list for columns
with (output_folder / 'strain_list.txt').open('r') as sh:
    strain_list = []
    for line in sh:
        strain_list.append(line.strip())
        
# Convert to dataframe (for now)
df1 = pd.DataFrame.from_dict(dict(dfp1)).T
df2 = pd.Series(dict(dfp2))
df1.columns = strain_list

count    90667.000000
mean        39.616729
std         17.794168
min          1.000000
25%         27.000000
50%         27.000000
75%         61.000000
max         72.000000
dtype: float64

In [39]:
f1 = pathlib.Path('/home/mladen/Strainr/test_R1.fastq')
f2 = pathlib.Path('/home/mladen/Strainr/test_R2.fastq')

# Use final results to get the top strain candidates

In [26]:
top_strain_indices = list(df2.value_counts().index[:5])
top_strain_names = [strain_list[i] for i in top_strain_indices]
top_strain_names

['GCA_000027305.1 Haemophilus influenzae Rd KW20',
 'GCA_000698365.1 Haemophilus influenzae CGSHiCZ412602',
 'GCA_000968335.1 Haemophilus influenzae 2019',
 'GCA_000197875.1 Haemophilus influenzae F3031',
 'GCA_000016465.1 Haemophilus influenzae PittEE']

# New code for multiple bins

In [44]:
def bin_helper(top_strain_names, df1, f1, f2, outdir):
    """ strain_dict is the strain_id : set of reads"""

    # Make a directory for output
    bin_dir = outdir / 'bins'
    bin_dir.mkdir(exist_ok=True,parents=True)

    nbins = 2
    print(f"Generating sequence files for the top {nbins} strains.")
    # most_reads_frame = (df1 != 0).sum().sort_values(ascending=False)

    procs, saved_strains = [], []
    for strain_id in top_strain_names:
        if strain_id == "NA":
            continue

        # Keep track of binned strains
        saved_strains.append(strain_id)
        print(f"Binning reads for {strain_id}...")
        p = mp.Process( target=bin_single, args=( strain_id, df1, f1, f2, output_folder, ),)
        p.start()
        procs.append(p)

    [p.join() for p in procs]

    # Return the set of strains and the list of processes
    return set(saved_strains), procs

bin_helper( top_strain_names,df1, f1, f2, output_folder)

Generating sequence files for the top 2 strains.
Binning reads for GCA_000027305.1 Haemophilus influenzae Rd KW20...
Binning reads for GCA_000698365.1 Haemophilus influenzae CGSHiCZ412602...
Binning reads for GCA_000968335.1 Haemophilus influenzae 2019...
Binning reads for GCA_000197875.1 Haemophilus influenzae F3031...
Binning reads for GCA_000016465.1 Haemophilus influenzae PittEE...
Saved 209122 records from /home/mladen/Strainr/test_R1.fastq to howdy4/bin.GCA_000968335.1_R1.fastq
Saved 268345 records from /home/mladen/Strainr/test_R1.fastq to howdy4/bin.GCA_000027305.1_R1.fastq
Saved 267901 records from /home/mladen/Strainr/test_R1.fastq to howdy4/bin.GCA_000016465.1_R1.fastq
Saved 271668 records from /home/mladen/Strainr/test_R1.fastq to howdy4/bin.GCA_000698365.1_R1.fastq
Saved 317174 records from /home/mladen/Strainr/test_R1.fastq to howdy4/bin.GCA_000197875.1_R1.fastq
Saved 209122 records from /home/mladen/Strainr/test_R2.fastq to howdy4/bin.GCA_000968335.1_R2.fastq
Saved 26834

({'GCA_000016465.1 Haemophilus influenzae PittEE',
  'GCA_000027305.1 Haemophilus influenzae Rd KW20',
  'GCA_000197875.1 Haemophilus influenzae F3031',
  'GCA_000698365.1 Haemophilus influenzae CGSHiCZ412602',
  'GCA_000968335.1 Haemophilus influenzae 2019'},
 [<Process name='Process-6' pid=449648 parent=448350 stopped exitcode=0>,
  <Process name='Process-7' pid=449649 parent=448350 stopped exitcode=0>,
  <Process name='Process-8' pid=449650 parent=448350 stopped exitcode=0>,
  <Process name='Process-9' pid=449651 parent=448350 stopped exitcode=0>,
  <Process name='Process-10' pid=449652 parent=448350 stopped exitcode=0>])

# Single bin

In [45]:

def bin_single( strain , df, forward_file, reverse_file, bin_dir):
    """ Given a strain, use hit info to extract reads"""
    strain_accession = strain.split()[0]
    paired_files = [forward_file,reverse_file]

    for fidx, input_file in enumerate(paired_files): # (0,R1), (1,R2)
        writefile_name = f"bin.{strain_accession}_R{fidx+1}.fastq"
        writefile = bin_dir / writefile_name

        if fidx == 0:
            reads = set(df[df[strain] > 0].index)
        else:
            reads = set(df[df[strain] > 0].index.str.replace('/1','/2'))
            
        with input_file.open('r') as original, writefile.open('w') as newfasta:
            records = (r for r in SeqIO.parse(original,'fastq') if r.id in reads)
            count = SeqIO.write(records, newfasta, "fastq")

        print(f"Saved {count} records from {input_file} to {writefile}")

    return
bin_single(strain_to_bin , df1, f1, f2, output_folder)

NameError: name 'strain_to_bin' is not defined

## Selecting strains the previous way - getting number of reads associated with each strain

In [38]:
import numpy as np
(df1 != 0).sum().sort_values(ascending=False)

# Annotate the old code

In [ ]:
def write_strain_fastas( strain_dict, f1, f2,):

    """ strain_dict is the strain_id : set of reads"""

    # Make a directory for output
    fastadir = outdir / 'fastas'
    fastadir.mkdir(exist_ok=True,parents=True)

    # Get the number of bins to perform
    max_n = len(topstrains)
    # print(f"Generating sequence files for the top {max_n} strains.")

    # Sort and select the top bins from the sorted list
    # strain_dict = dict(sorted(strain_dict.items(), key=lambda item: item[1])[:max_n])

    # Loop through each strain to make the file
    procs, saved_strains = [], []
    for strain_id, read_set in strain_dict.items():

        # Don't bin NA
        if strain_id == "NA":
            continue
        
        # Keep track of binned strains
        saved_strains.append(strain_id)
        print(f"Writing {len(read_set)} reads for {strain_id}...")

        # Call the process with dict key + val, file1 + file2, output
        p = Process( target=mp_write_single_strain, args=( strain_id, read_set, f1, f2, fastadir, ),)

        # Begin the process
        p.start()
        
        # Make a process list
        procs.append(p)

    # Collect processes
    # [p.join() for p in procs]

    # Return the set of strains and the list of processes
    return set(saved_strains), procs
    
    
def mp_write_single_strain( strain , read_set, forward_file, reverse_file,  fastadir , ):

    paired_files = [forward_file,reverse_file]
    fext = "fastq"

    # Loop through forward and reverse forward=0, reverse=1
    for fidx, input_file in enumerate(paired_files): # (0,R1), (1,R2)

        # Set up list of reads + string-replace
        if fidx == 1: #reverse
            read_names = set( read.replace("/1", "/2") for read in read_set)
        else:
            read_names = set(read for read in read_set)


        # Set up output file
        writefile_name = f"sbin.{strain}_R{fidx+1}.{fext}"  
        writefile = fastadir / writefile_name

        # Open input, find reads that match dict, write to strain_fasta

        with _open(input_file) as rhandle, open(writefile, "w") as whandle:
            # Open original fasta, open future fasta

            # Collect SeqIO records (id,dna,qual) that are in the read_set
            records = (r for r in SeqIO.parse(rhandle,'fastq') if r in read_names)
            # write them to a file
            count = SeqIO.write(records, whandle, "fastq")
            print(f"Saved {count} records from {input_file} to {writefile}")

    return


# Old Code

In [ ]:

class Binner:
    def __init__(self, count_table, min_hit_threshold, min_k_threshold, final_hits,f1, f2):
        self.table = count_table
        self.min_hit = min_hit_threshold
        self.min_k = min_k_threshold
        self.final_hits = final_hits
        self.f1 = f1
        self.f2 = f2
        
        
        
        

def filter_strains_bin(strain_map:dict, minhits: int,fhits: dict):
    return { k: v for k, v in strain_map.items() 
            if len(fhits[k]) > minhits 
            }

def bin_stuff(intermediate, fhits):
    minhits = 1000
    df = pd.read_results('intermed.csv').set_index('Unnamed: 0')

    strain_map = strain_to_reads_map(intermediate, min_k=1)
    print(strain_map)
    strain_map = filter_strains_bin(strain_map, minhits, fhits)
    print(strain_map)
    topstrains = select_top_bins(df,nstrains,min_k)
    print(topstrains)
    f2 = params['reverse']
    print(f2)
    astrain_set, aplist = write_strain_fastas( strain_map, f1, f2)
    print(astrain_set)
    print(aplist)
    [ap.join() for ap in aplist]
    return

def select_top_bins(df,nstrains,min_k):
    min_k= 90
    nstrains = 5
    strain_nhits = {}
    for strain in df.columns:
        nreads = len(list(df[df[strain] > min_k].index))
        strain_nhits[strain] = nreads
    #     print(f"The strain {strain} has {nreads} reads with greater than {min_k} k-mer hits") 
    topstrains = sorted(strain_nhits.items(),key=lambda kv: kv[1],reverse=True)[:nstrains]
    return [i[0] for i in topstrains]

def write_strain_fastas( strain_dict, f1, f2,):
    fastadir = outdir / 'fastas'
    fastadir.mkdir(exist_ok=True,parents=True)
    max_n = len(topstrains)

    # print(f"Generating sequence files for the top {max_n} strains.")
    # strain_dict = dict(sorted(strain_dict.items(), key=lambda item: item[1])[:max_n])

    # Loop through each strain to make the file
    procs, saved_strains = [], []
    for strain_id, read_set in strain_dict.items():
        if strain_id == "NA":
            continue

        saved_strains.append(strain_id)
        print(f"Writing {len(read_set)} reads for {strain_id}...")
        p = Process( target=mp_write_single_strain, args=( strain_id, read_set, f1, f2, fastadir, ),)
        p.start()
        procs.append(p)

    # [p.join() for p in procs]
    return set(saved_strains), procs

    
    
def mp_write_single_strain( strain , read_set, forward_file, reverse_file,  fastadir , ):

    paired_files = [forward_file,reverse_file]
    fext = "fastq"
    for fidx, input_file in enumerate(paired_files): # (0,R1), (1,R2)
        if fidx == 1: #reverse
            read_names = set( read.replace("/1", "/2") for read in read_set)
        else:
            read_names = set(read for read in read_set)

        writefile_name = f"sbin.{strain}_R{fidx+1}.{fext}"  
        writefile = fastadir / writefile_name

        # Open input, find reads that match dict, write to strain_fasta
        with _open(input_file) as rhandle, open(writefile, "w") as whandle:
            records = (r for r in SeqIO.parse(rhandle,'fastq') if r in read_names)
            count = SeqIO.write(records, whandle, "fastq")
            print(f"Saved {count} records from {input_file} to {writefile}")

    return


"""
import pandas as pd
df = pd.read_csv('intermed.csv').set_index('Unnamed: 0')

        
def select_top_bins(df,nstrains,min_k):
    min_k= 90
    nstrains = 5
    strain_nhits = {}
    for strain in df.columns:
        nreads = len(list(df[df[strain] > min_k].index))
        strain_nhits[strain] = nreads
    #     print(f"The strain {strain} has {nreads} reads with greater than {min_k} k-mer hits") 
    topstrains = sorted(strain_nhits.items(),key=lambda kv: kv[1],reverse=True)
    topstrains = topstrains[:nstrains]
    return [i[0] for i in topstrains]

        
"""